### Convert intersections described in Citations dataset into gis coordinates to be able to do spatial analysis with the Citations dataset

#### Using the City of Scottsdale **Streets** Dataset I was able to access the gis coordinates for all its streets.  For the north/south streets I was planning on using the street latitude to represent every intersection of that street with every other cross street.  Since Scottsdale streets are aligned mostly north/south and east/west, I believe this will give me a roughly accurate location for each cross street intersection.

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import pyodbc
import pandas as pd
import seaborn as sns
import datetime
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# databaseConnection = 'DRIVER={SQL Server}; SERVER=ServerName; Database=DatbaseName; UID=UserId; PWD=password;'
connection = pyodbc.connect('DRIVER={SQL Server};SERVER=WINDOWS-A6S01I2\SQLEXPRESS;DATABASE=City_of_Scottsdale;Trusted_Connection=yes;')
cursor = connection.cursor()

# SQL query to import all citation data
citations = '''
  select * from [dbo].[spd_PDCitations$]  
  '''
citations = pd.read_sql(citations, connection)
connection.close()

In [3]:
pd.set_option('display.max_columns', None)
citations.head()

,tcmainid,Citation #,Citation Date,Citation Time,Arizona Statute Code,Charge Description,Citation Type Code,Citation Type Description,Officer Badge #,Street Number,Street,City,State,Zip,District,Beat,Zone,Cited Person,Cited Person Age,Cited Person Race,Cited Person Sex,Cited Person Ethnicity
0,621784.0,2065379.0,2018-06-23,204.0,13-1203A1,Assault-Intentional/Recklessly Injure,CR,Criminal,1489.0,1xxx,N Granite Reef Rd,Scottsdale,AZ,NaN,D1,2.0,412.0,"Hill, Julie",25.0,I,F,N
1,621797.0,2063172.0,2018-06-23,928.0,13-3415A,Drug Paraphernalia-Possess/Use,CR,Criminal,1214.0,7xxx,N Hayden Rd,Scottsdale,AZ,NaN,D3,10.0,2504.0,"Mc Cuan, Michael",51.0,W,M,N
2,621806.0,2056700.0,2018-06-23,1053.0,28-701A,Failure to Control Speed to Avoid A Collision,CV,Civil Traffic,1360.0,xxx,E Indian Bend Rd / N Scottsdale Rd,Scottsdale,AZ,NaN,D3,10.0,2501.0,"Glasscock, James",60.0,W,M,N
3,621847.0,2050143.0,2018-06-23,2345.0,19-19A,Urinate/Defecate In Public,MC,Municipal Code,1345.0,4xxx,N Saddlebag Tr,Scottsdale,AZ,NaN,D2,6.0,904.0,"Still, Marcus",22.0,B,M,N
4,621878.0,2063174.0,2018-06-24,927.0,28-3473A,Suspended/Revoked D.L.,CT,Criminal Traffic,1214.0,8xxx,E McDonald DR,Scottsdale,AZ,NaN,D2,8.0,2205.0,"Chacon, Kimberly",48.0,W,F,N


In [4]:
cross_streets_split = citations['Street'].str.split('/', n=1, expand=True)

In [5]:
cross_streets_split.head()

,0,1
0,N Granite Reef Rd,None
1,N Hayden Rd,None
2,E Indian Bend Rd,N Scottsdale Rd
3,N Saddlebag Tr,None
4,E McDonald DR,None


In [6]:
# insert split streets into citations dataset as two columns
citations['street_1'], citations['street_2'] = cross_streets_split[0], cross_streets_split[1]

In [7]:
citations.head()

,tcmainid,Citation #,Citation Date,Citation Time,Arizona Statute Code,Charge Description,Citation Type Code,Citation Type Description,Officer Badge #,Street Number,Street,City,State,Zip,District,Beat,Zone,Cited Person,Cited Person Age,Cited Person Race,Cited Person Sex,Cited Person Ethnicity,street_1,street_2
0,621784.0,2065379.0,2018-06-23,204.0,13-1203A1,Assault-Intentional/Recklessly Injure,CR,Criminal,1489.0,1xxx,N Granite Reef Rd,Scottsdale,AZ,NaN,D1,2.0,412.0,"Hill, Julie",25.0,I,F,N,N Granite Reef Rd,None
1,621797.0,2063172.0,2018-06-23,928.0,13-3415A,Drug Paraphernalia-Possess/Use,CR,Criminal,1214.0,7xxx,N Hayden Rd,Scottsdale,AZ,NaN,D3,10.0,2504.0,"Mc Cuan, Michael",51.0,W,M,N,N Hayden Rd,None
2,621806.0,2056700.0,2018-06-23,1053.0,28-701A,Failure to Control Speed to Avoid A Collision,CV,Civil Traffic,1360.0,xxx,E Indian Bend Rd / N Scottsdale Rd,Scottsdale,AZ,NaN,D3,10.0,2501.0,"Glasscock, James",60.0,W,M,N,E Indian Bend Rd,N Scottsdale Rd
3,621847.0,2050143.0,2018-06-23,2345.0,19-19A,Urinate/Defecate In Public,MC,Municipal Code,1345.0,4xxx,N Saddlebag Tr,Scottsdale,AZ,NaN,D2,6.0,904.0,"Still, Marcus",22.0,B,M,N,N Saddlebag Tr,None
4,621878.0,2063174.0,2018-06-24,927.0,28-3473A,Suspended/Revoked D.L.,CT,Criminal Traffic,1214.0,8xxx,E McDonald DR,Scottsdale,AZ,NaN,D2,8.0,2205.0,"Chacon, Kimberly",48.0,W,F,N,E McDonald DR,None


Next steps:
1. Identify streets with N/S identifyers at the front and match them up with that streets respective latitude
2. Identify streets with E/W identifiers at the front and match them up with that streets respective longitude
3. Plot several intersections based on longitude and latitude and spot check to verify the locations are reasonably close to the intersection identified by the streets